In [1]:
import io
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import boto3
import sagemaker
from sagemaker import get_execution_role

In [2]:
session = sagemaker.Session()
print(session)

role = get_execution_role()
print(role)

bucket = session.default_bucket()

arn:aws:iam::240038582877:role/service-role/AmazonSageMaker-ExecutionRole-20191028T202433


In [3]:
# load data
prefix = 'segmentation'

data_key= prefix + '/mailout_train_raw.csv'
data_location = 's3://{}/{}'.format(bucket, data_key)

In [4]:
tmp_list = []

for chunk in pd.read_csv(data_location, chunksize=100000):
    tmp_list.append(chunk)
    
train_df = pd.concat(tmp_list, axis=0).drop(['Unnamed: 0'], axis=1)
print(train_df.shape)
del tmp_list

/home/ec2-user/anaconda3/envs/mxnet_p36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(42962, 367)


In [5]:
#shuffle data
train_df = train_df.sample(frac=1, axis=1)

In [25]:
features = train_df.select_dtypes(['int', 'float']).columns.values[:100]
features = np.append(features, 'RESPONSE')
features

array(['KBA13_CCM_1000', 'D19_KONSUMTYP_MAX', 'KBA05_ANTG4',
       'KK_KUNDENTYP', 'D19_GESAMT_ANZ_24', 'KBA05_CCM3',
       'D19_BEKLEIDUNG_GEH', 'D19_FREIZEIT', 'SEMIO_FAM',
       'KBA13_CCM_2500', 'KBA13_AUTOQUOTE', 'KBA13_AUDI',
       'D19_DIGIT_SERV', 'DSL_FLAG', 'PLZ8_ANTG3', 'ALTER_KIND3',
       'KBA13_KRSSEG_VAN', 'KBA13_FAB_SONSTIGE', 'KBA13_KW_50',
       'D19_VERSI_DATUM', 'KBA13_SEG_MINIVANS', 'D19_BANKEN_DIREKT',
       'KBA13_BJ_2009', 'KBA13_KW_0_60', 'KBA13_KMH_110',
       'KBA13_CCM_2501', 'KBA13_ALTERHALTER_30', 'LP_STATUS_FEIN',
       'STRUKTURTYP', 'KBA05_MOD3', 'NATIONALITAET_KZ', 'SEMIO_LUST',
       'FINANZ_UNAUFFAELLIGER', 'KBA05_ANTG2', 'KBA05_SEG9',
       'FINANZ_MINIMALIST', 'KBA13_ALTERHALTER_45', 'VHA', 'SOHO_KZ',
       'KBA05_KW3', 'D19_VERSAND_DATUM', 'LP_STATUS_GROB',
       'D19_GESAMT_ANZ_12', 'KKK', 'W_KEIT_KIND_HH', 'SEMIO_KAEM',
       'KBA13_KRSZUL_NEU', 'GEBAEUDETYP', 'CJT_TYP_1',
       'KBA13_SEG_MITTELKLASSE', 'KBA05_KW1', 'KBA13_KMH_21

In [26]:
df = train_df[features]
df.shape

(42962, 101)

In [27]:
y = df['RESPONSE']
X = df.drop(['RESPONSE'], axis=1)

In [28]:
y.dtypes

dtype('int64')

In [29]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X)
StandardScaler()
transformed_df = pd.DataFrame(columns = X.columns.values, index = X.index.values, data=scaler.transform(X));

/home/ec2-user/anaconda3/envs/mxnet_p36/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/ec2-user/anaconda3/envs/mxnet_p36/lib/python3.6/site-packages/ipykernel/__main__.py:5: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.


In [30]:
transformed_df = transformed_df.fillna(0)
transformed_df.isnull().any(axis=0);
transformed_df.shape

(42962, 100)

In [31]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(multi_class='ovr').fit(transformed_df,y)

/home/ec2-user/anaconda3/envs/mxnet_p36/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [32]:
#Return the mean accuracy on the given test data and labels
clf.predict_proba(transformed_df)

array([[0.97840254, 0.02159746],
       [0.99380316, 0.00619684],
       [0.99050343, 0.00949657],
       ...,
       [0.97409925, 0.02590075],
       [0.99223352, 0.00776648],
       [0.99496613, 0.00503387]])

In [33]:
clf.score(transformed_df, y)

0.9876169638284996